In [ ]:
import holoviews as hv
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn
import param

In [ ]:
%load_ext autoreload
%autoreload 2
import spectra
import util

In [ ]:
hv.extension("bokeh")

# Spectral response

In [ ]:
bins = np.arange(300, 1000)

In [ ]:
%%time
sources = {"SOLIS-3C": "7016", "SOLIS-565C": "7004"}

dichroics = {
    "Chroma ZT514rdc": "560",
    "Chroma ZT532rdc": "645",
    "Chroma T550lpxr": "803",
    "Chroma T556lpxr": "658",
    "Chroma ZT561rdc": "523",
    "Chroma ZT561rdc-xr": "708",
    "Chroma ZT568rdc": "604",
    "Chroma T570lpxr": "427",
    "Chroma T590lpxr": "593",
    "Chroma ZT594rdc": "569",
    "Chroma T600lpxr": "609",
    "Chroma T610lpxr": "601",
    "Chroma ZT633rdc": "668",
    "Chroma T635lpxr": "616",
    "Chroma ZT640rdc": "439",
}

longpass_filters = {
    "Chroma ET520LP": "760",
    "Chroma ET525lp": "530",
    "Chroma ET542lp": "461",
    "Chroma ET570lp": "487",
    "Chroma ET575lp": "620",
    "Chroma ET590lp": "805",
    "Chroma ET610lp": "350",
    "Chroma RET638lp": "2855",
    "Chroma ET655lp": "683",
    "Chroma ET665lp": "573",
}

sources = {
    name: spectra.get_fpbase_spectrum(q, bins=bins) for name, q in sources.items()
}
dichroics = {
    name: spectra.get_fpbase_spectrum(q, bins=bins) for name, q in dichroics.items()
}
longpass_filters = {
    name: spectra.get_fpbase_spectrum(q, bins=bins)
    for name, q in longpass_filters.items()
}

In [ ]:
%%time
semrock_dichroic_urls = [
    "https://www.semrock.com/filtersRefined.aspx?minWL=520&maxWL=660&id=497,800&page=1&so=0&recs=10000"
]
semrock_longpass_urls = [
    f"https://www.semrock.com/filtersRefined.aspx?minWL=520&maxWL=660&id=21,{id_}&page=1&so=0&recs=10000"
    for id_ in (537, 538)  # RazorEdge ultrasharp (not available in 32mm standard): 545
]

dichroics = {
    **dichroics,
    **spectra.get_semrock_spectra(semrock_dichroic_urls, bins=bins),
}
longpass_filters = {
    **longpass_filters,
    **spectra.get_semrock_spectra(semrock_longpass_urls, bins=bins),
}

In [ ]:
dichroics = dict(
    sorted(dichroics.items(), key=lambda x: (x[1].loc[:700] < 0.5)[::-1].idxmax())
)
longpass_filters = dict(
    sorted(longpass_filters.items(), key=lambda x: (x[1].loc[:700] < 0.5).idxmax())
)

In [ ]:
%%time
fps = spectra.get_fpbase_protein_spectra(bins=bins)

## Filters

In [ ]:
def combine_filter(dc, lp, threshold=0.5, od_threshold=6, max_wavelength=700):
    dc = dc.loc[:max_wavelength]
    lp = lp.loc[:max_wavelength]
    dc_od = -np.log10(dc)
    lp_od = -np.log10(lp)
    # highest wavelength that gives <90% excitation (dichroic reflectance)
    ex_cutoff = (dc < 1 - threshold)[::-1].idxmax()
    # highest wavelength that gives >OD6 rejection
    ex_od_cutoff = (lp_od > od_threshold)[::-1].idxmax()
    # highest wavelength that gives <90% transmission (dichroic+lp)
    em_cutoff = (dc_od + lp_od < -np.log10(threshold)).idxmax()
    gap = em_cutoff - ex_cutoff
    ex_gap = ex_od_cutoff - ex_cutoff
    # highest OD below excitation cutoff
    worst_rejection = (dc_od + lp_od).loc[:ex_cutoff].dropna().min()
    return pd.Series(
        {
            "ex_cutoff": ex_cutoff,
            "ex_od_cutoff": ex_od_cutoff,
            "em_cutoff": em_cutoff,
            "gap": gap,
            "ex_gap": ex_gap,
            "worst_rejection": worst_rejection,
        }
    )


filter_combinations = pd.concat(
    {
        dc_name: pd.concat(
            {
                lp_name: combine_filter(dc_spectrum, lp_spectrum)
                for lp_name, lp_spectrum in longpass_filters.items()
            },
            axis=1,
        ).T
        for dc_name, dc_spectrum in dichroics.items()
    }
).sort_values(["ex_cutoff", "gap", "ex_gap"])

In [ ]:
acceptable_filter_combinations = filter_combinations[
    (filter_combinations["gap"] < 10) & (filter_combinations["ex_gap"].abs() < 20)
]

In [ ]:
acceptable_filter_combinations

In [ ]:
# for each dichroic, pick the longpass that minimizes the gap
selected_filter_combinations = (
    acceptable_filter_combinations.groupby(level=0)
    .apply(lambda x: x.loc[[x["ex_gap"].abs().idxmin()]])
    .droplevel(0)
    .sort_values("ex_cutoff")
    .index.values
)
# selected_filter_combinations = acceptable_filter_combinations.index

In [ ]:
acceptable_filter_combinations.loc[selected_filter_combinations]

## FPs

In [ ]:
def fp_efficiency(dc, lp, ex, em, ex_cutoff, em_cutoff):
    return pd.Series(
        {
            ("efficiency", "ex"): ((1 - dc) * (ex / ex.sum())).sum(),
            ("efficiency", "em"): (dc * lp * (em / em.sum())).sum(),
            ("cutoff_amplitude", "ex"): ex.loc[ex_cutoff],
            ("cutoff_amplitude", "em"): em.loc[em_cutoff],
            ("margin", "ex"): ex_cutoff - ex.idxmax(),
            ("margin", "em"): em.idxmax() - em_cutoff,
        }
    )


def evaluate_filter_combinations(
    filter_combo_names, fp_names, dichroics, longpass_filters, filter_combinations, fps
):
    d = {}
    for filter_combo in filter_combo_names:
        dc = dichroics[filter_combo[0]]
        lp = longpass_filters[filter_combo[1]]
        dd = {}
        for fp_name in fp_names:
            ex = fps[fp_name]["spectra"]["ex"]
            em = fps[fp_name]["spectra"]["em"]
            dd[fp_name] = (
                fp_efficiency(
                    dc,
                    lp,
                    ex,
                    em,
                    filter_combinations.loc[filter_combo]["ex_cutoff"],
                    filter_combinations.loc[filter_combo]["em_cutoff"],
                )
                .to_frame()
                .T
            )
        d[filter_combo] = (
            pd.concat(dd, axis=1)
            .reorder_levels([1, 2, 0], axis=1)
            .sort_index(axis=1, level=1, sort_remaining=False)
        )
    return pd.concat(d, axis=0).droplevel(-1)

In [ ]:
selected_fps = [
    "mScarlet-I",
    "mScarlet-H",
    "mCherry",
    "mCherry2",
    "mKate2",
    "E2-Crimson",
    "TurboRFP",
]
selected_fps = sorted(selected_fps, key=lambda x: fps[x]["ex_max"])
filter_metrics = evaluate_filter_combinations(
    selected_filter_combinations,
    selected_fps,
    dichroics,
    longpass_filters,
    filter_combinations,
    fps,
)

In [ ]:
def spectra_viewer(fp_names=None, dc_names=None, lp_names=None):
    viewer = spectra.SpectraViewer(
        fps, dichroics, longpass_filters, fp_names, dc_names, lp_names
    )
    return pn.Row(pn.Column("## Spectra", viewer.param), viewer.view)

In [ ]:
spectra.show_heatmap(filter_metrics["efficiency"])

In [ ]:
spectra.show_heatmap(filter_metrics["cutoff_amplitude"])

In [ ]:
spectra.show_heatmap(filter_metrics["margin"], vmin=0, highlight_negative="black")

In [ ]:
spectra_viewer(fp_names=selected_fps)

In [ ]:
", ".join(fps.keys())